In [1]:
import numpy as np
from osim.env import ProstheticsEnv
from baselines.common.vec_env.subproc_vec_env import SubprocVecEnv
import torch
import torch.nn as nn
import pickle

In [2]:
nproc = 32

In [3]:
def make_env(seed):
    def _f():
        env = ProstheticsEnv(visualize=False)
        env.change_model(model='3D', prosthetic=True, difficulty=0, seed=seed)
        return env
    return _f

In [4]:
envs = [make_env(seed) for seed in range(nproc)]
envs = SubprocVecEnv(envs)
obs = envs.reset()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.

In [5]:
D_in=envs.observation_space.shape[0]
D_out=envs.action_space.shape[0]
H=1024
device = torch.device('cuda')

In [6]:
random_policy = torch.nn.Sequential(
                torch.nn.Linear(D_in, H),
                torch.nn.ReLU(),
                torch.nn.Linear(H, H),
                torch.nn.ReLU(),
                torch.nn.Linear(H, H),
                torch.nn.ReLU(),
                torch.nn.Linear(H, D_out),
                ).to(device)

In [9]:
def data_collection(j):    
    dataset=list()
    for i in range(j):
        done=False
        obs = envs.reset()
        while not done:
            action=random_policy(torch.from_numpy(obs).float().cuda()).cpu().data.numpy()
            obs_next,_,done,_=envs.step(action)
            dataset.append([obs,action,obs_next])
            obs=obs_next
            done=any(done)
        print(len(dataset))
    pickle.dump(dataset,open('data_haris_'+str(j)+'.pkl','wb'))

In [ ]:
[data_collection(j) for j in range(10)]

In [ ]:
pickle.dump(dataset,open('data_haris.pkl','wb'))

In [ ]:
data_collection(100)

111
222
